In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
from pyspark import SparkContext, SparkConf
sc = SparkContext('local')
print(sc.version)

22/04/18 21:12:45 WARN Utils: Your hostname, MSI resolves to a loopback address: 127.0.1.1; using 172.28.44.162 instead (on interface eth0)
22/04/18 21:12:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/18 21:12:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


3.2.1


In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
# Print my_spark
print(spark)

In [4]:
# Print the tables in the catalog
print(spark.catalog.listTables())

[]


In [5]:
# Create pd_temp
pd_temp = pd.DataFrame(np.random.random(10))

# Create spark_temp from pd_temp
spark_temp = spark.createDataFrame(pd_temp)

# Examine the tables in the catalog
print(spark.catalog.listTables())

# Add spark_temp to the catalog
spark_temp.createOrReplaceTempView('temp')

# Examine the tables in the catalog again
print(spark.catalog.listTables())

[]
[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [7]:
# Read in the airports data
airports = spark.read.csv(['/mnt/c/Users/lokes/Downloads/airports.csv'], header = True)

# Show the data
airports.show(10)

+---------+--------------------+-------------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-------------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|    Allentown|   PA|    USA|40.65236| -75.44040|
|      ABI|Abilene Regional ...|      Abilene|   TX|    USA|32.41132| -99.68190|
|      ABQ|Albuquerque Inter...|  Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|     Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|       Albany|   GA|    USA|31.53552| -84.19447|
|      ACK|Nantucket Memoria...|    Nantucket|   MA|    USA|41.25305| -70.06018|
|      ACT|Waco Regional Air...|         Waco|   TX|    USA|31.61129| -97.23052|
|      ACV|      Arcata Airport|Arcata/Eureka|   CA|    USA|40.97812|-124.10862|
|      ACY|Atlantic City Int...|Atlantic City|   NJ|    USA|39.45758| -74.57717|
|      ADK|        Adak Airp

In [9]:
# Read in the airports data
flights = spark.read.csv(['/mnt/c/Users/lokes/Downloads/flights.csv'], header = True)

# Show the data shape
print((flights.count(), len(flights.columns)))

# print the tables in catalog
print(spark.catalog.listTables())

# adding data into spark view for sql querying
flights.createOrReplaceTempView('flights')

# print the tables in catalog
print(spark.catalog.listTables())

22/04/18 21:15:54 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


(5819079, 31)
[Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]
[Table(name='flights', database=None, description=None, tableType='TEMPORARY', isTemporary=True), Table(name='temp', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]


In [10]:
# see all columns in the table 
print(flights.columns)

['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']


In [11]:
# Don't change this query
query = "SELECT AIRLINE, FLIGHT_NUMBER, TAIL_NUMBER, ORIGIN_AIRPORT, DESTINATION_AIRPORT, SCHEDULED_DEPARTURE FROM flights LIMIT 5"

# Get the first 10 rows of flights
flights5 = spark.sql(query)

# Show the results
flights5.show()

+-------+-------------+-----------+--------------+-------------------+-------------------+
|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|
+-------+-------------+-----------+--------------+-------------------+-------------------+
|     AS|           98|     N407AS|           ANC|                SEA|               0005|
|     AA|         2336|     N3KUAA|           LAX|                PBI|               0010|
|     US|          840|     N171US|           SFO|                CLT|               0020|
|     AA|          258|     N3HYAA|           LAX|                MIA|               0020|
|     AS|          135|     N527AS|           SEA|                ANC|               0025|
+-------+-------------+-----------+--------------+-------------------+-------------------+



In [12]:
# Don't change this query
query = "SELECT ORIGIN_AIRPORT, DESTINATION_AIRPORT, COUNT(*) as N FROM flights GROUP BY ORIGIN_AIRPORT, DESTINATION_AIRPORT"

# Run the query
flight_counts = spark.sql(query)

# Convert the results to a pandas DataFrame
pd_counts = flight_counts.toPandas()

# Print the head of pd_counts
print(pd_counts.head())

  ORIGIN_AIRPORT DESTINATION_AIRPORT     N
0            BQN                 MCO   441
1            PHL                 MCO  4869
2            MCI                 IAH  1698
3            SPI                 ORD   998
4            SNA                 PHX  3846


In [14]:
# Create the DataFrame flights
flights = spark.table("flights")

# Add duration_hrs
flights = flights.withColumn('duration_hrs', flights.AIR_TIME/60.)

# Show the head
flights.select('duration_hrs').show(10)

+------------------+
|      duration_hrs|
+------------------+
| 2.816666666666667|
| 4.383333333333334|
| 4.433333333333334|
|               4.3|
| 3.316666666666667|
| 3.433333333333333|
| 2.566666666666667|
|               3.8|
|2.8833333333333333|
|               3.1|
+------------------+
only showing top 10 rows



In [15]:
# Filter flights by passing a string
long_flights1 = flights.filter("DISTANCE > 1000")

# Filter flights by passing a column of boolean values
long_flights2 = flights.filter(flights.DISTANCE > 1000)


In [16]:
# Select the first set of columns
selected1 = flights.select('TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',)

# Select the second set of columns
temp = flights.select(flights.ORIGIN_AIRPORT, flights.DESTINATION_AIRPORT, flights.AIRLINE)

temp.show()

+--------------+-------------------+-------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|AIRLINE|
+--------------+-------------------+-------+
|           ANC|                SEA|     AS|
|           LAX|                PBI|     AA|
|           SFO|                CLT|     US|
|           LAX|                MIA|     AA|
|           SEA|                ANC|     AS|
|           SFO|                MSP|     DL|
|           LAS|                MSP|     NK|
|           LAX|                CLT|     US|
|           SFO|                DFW|     AA|
|           LAS|                ATL|     DL|
|           DEN|                ATL|     DL|
|           LAS|                MIA|     AA|
|           LAX|                MSP|     DL|
|           SLC|                ATL|     DL|
|           SEA|                MSP|     DL|
|           ANC|                SEA|     AS|
|           ANC|                SEA|     DL|
|           SFO|                IAH|     UA|
|           ANC|                PDX|     AS|
|         

In [17]:
# Define first filter
filterA = flights.ORIGIN_AIRPORT == "SEA"

# Define second filter
filterB = flights.DESTINATION_AIRPORT == "PDX"

# Filter the data, first by filterA then by filterB
selected2 = temp.filter(filterA).filter(filterB)

In [18]:
# Define avg_speed
avg_speed = (flights.DISTANCE/(flights.AIR_TIME/60)).alias("avg_speed")

# Select the correct columns
speed1 = flights.select('TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', avg_speed)

# Create the same table using a SQL expression
speed2 = flights.selectExpr('TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', "DISTANCE/(AIR_TIME/60) as avg_speed")

In [19]:
#Cast the columns to integers
flights = flights.withColumn("MONTH", flights.MONTH.cast("integer"))
flights = flights.withColumn("DAY_OF_WEEK", flights.DAY_OF_WEEK.cast("integer"))
flights = flights.withColumn("AIR_TIME", flights.AIR_TIME.cast("integer"))
flights = flights.withColumn("DISTANCE", flights.DISTANCE.cast("double"))
flights = flights.withColumn("ARRIVAL_DELAY", flights.ARRIVAL_DELAY.cast("integer"))

In [20]:
# Find the shortest flight from PDX in terms of distance
flights.filter(flights.ORIGIN_AIRPORT == 'PDX').groupBy().min('DISTANCE').show()

# Find the longest flight from SEA in terms of air time
flights.filter(flights.ORIGIN_AIRPORT == 'SEA').groupBy().max('AIR_TIME').show()

+-------------+
|min(DISTANCE)|
+-------------+
|        129.0|
+-------------+



+-------------+
|max(AIR_TIME)|
+-------------+
|          412|
+-------------+



In [21]:
# Group by tailnum
by_plane = flights.groupBy("TAIL_NUMBER")

# Number of flights each plane made
by_plane.count().show(10)

# Group by origin
by_origin = flights.groupBy("ORIGIN_AIRPORT")

# Average duration of flights from PDX and SEA
by_origin.avg("AIR_TIME").show(10)

+-----------+-----+
|TAIL_NUMBER|count|
+-----------+-----+
|     N38451|  946|
|     N567AA| 1458|
|     N623NK| 1669|
|     N442AS| 1210|
|     N902DE| 1704|
|     N4YUAA| 1290|
|     N466SW| 1942|
|     N516UA|  439|
|     N866AS| 2397|
|     N499AA| 1292|
+-----------+-----+
only showing top 10 rows



+--------------+------------------+
|ORIGIN_AIRPORT|     avg(AIR_TIME)|
+--------------+------------------+
|           BGM|  65.7915057915058|
|           PSE|182.11517615176152|
|           INL|31.967625899280577|
|           MSY|102.93938681708973|
|           PPG|  309.188679245283|
|           GEG| 84.57923728813559|
|           SNA|106.89280265882097|
|           BUR| 68.00404814594916|
|           GRB|48.289618644067794|
|           GTF| 83.24794238683127|
+--------------+------------------+
only showing top 10 rows



In [22]:
#Import pyspark.sql.functions as F
import pyspark.sql.functions as F

# cast
flights = flights.withColumn("DEPARTURE_DELAY", flights.DEPARTURE_DELAY.cast("integer"))

# Group by month and dest
by_month_dest = flights.groupBy('MONTH', 'DESTINATION_AIRPORT')

# Average departure delay by month and destination
by_month_dest.avg('DEPARTURE_DELAY').show(10)

# Standard deviation of departure delay
by_month_dest.agg(F.stddev('DEPARTURE_DELAY')).show(10)

+-----+-------------------+--------------------+
|MONTH|DESTINATION_AIRPORT|avg(DEPARTURE_DELAY)|
+-----+-------------------+--------------------+
|    1|                ACY|    8.33433734939759|
|    1|                EYW|   6.551440329218107|
|    1|                OME|                0.05|
|    1|                RDM|   9.701863354037267|
|    1|                TWF|  -4.711864406779661|
|    2|                AVP|  11.623529411764705|
|    3|                LNK|    8.91304347826087|
|    3|                BJI| -1.0806451612903225|
|    1|                AEX|    4.98932384341637|
|    1|                GNV|   6.982062780269058|
+-----+-------------------+--------------------+
only showing top 10 rows



+-----+-------------------+----------------------------+
|MONTH|DESTINATION_AIRPORT|stddev_samp(DEPARTURE_DELAY)|
+-----+-------------------+----------------------------+
|    1|                ACY|           32.81265545745788|
|    1|                EYW|           63.65052137951282|
|    1|                OME|          19.546923118225806|
|    1|                RDM|          34.260918828983726|
|    1|                TWF|           7.985009333135624|
|    2|                AVP|           25.87606748227672|
|    3|                LNK|           40.74236129383307|
|    3|                BJI|          13.324817701944676|
|    1|                AEX|          21.823559351202647|
|    1|                GNV|           27.69378310655295|
+-----+-------------------+----------------------------+
only showing top 10 rows



In [23]:
print(airports.columns)

# Examine the data
print(airports.show(10))

['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE']
+---------+--------------------+-------------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-------------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|    Allentown|   PA|    USA|40.65236| -75.44040|
|      ABI|Abilene Regional ...|      Abilene|   TX|    USA|32.41132| -99.68190|
|      ABQ|Albuquerque Inter...|  Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|     Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|       Albany|   GA|    USA|31.53552| -84.19447|
|      ACK|Nantucket Memoria...|    Nantucket|   MA|    USA|41.25305| -70.06018|
|      ACT|Waco Regional Air...|         Waco|   TX|    USA|31.61129| -97.23052|
|      ACV|      Arcata Airport|Arcata/Eureka|   CA|    USA|40.97812|-124.10862|
|      ACY|Atlantic City Int...

In [24]:
# Rename the faa column
airports = airports.withColumnRenamed("IATA_CODE", "DESTINATION_AIRPORT")

# Join the DataFrames
flights_with_airports = flights.join(airports , on = 'DESTINATION_AIRPORT', how = 'leftouter')

# Examine the new DataFrame
print(flights_with_airports.columns)
print(flights_with_airports.count())

['DESTINATION_AIRPORT', 'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY', 'duration_hrs', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE']


5819079


In [25]:
flights_with_airports.select('FLIGHT_NUMBER', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE').show(10)

+-------------+--------------------+-----------------+-----+-------+--------+----------+
|FLIGHT_NUMBER|             AIRPORT|             CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+-------------+--------------------+-----------------+-----+-------+--------+----------+
|           98|Seattle-Tacoma In...|          Seattle|   WA|    USA|47.44898|-122.30931|
|         2336|Palm Beach Intern...|  West Palm Beach|   FL|    USA|26.68316| -80.09559|
|          840|Charlotte Douglas...|        Charlotte|   NC|    USA|35.21401| -80.94313|
|          258|Miami Internation...|            Miami|   FL|    USA|25.79325| -80.29056|
|          135|Ted Stevens Ancho...|        Anchorage|   AK|    USA|61.17432|-149.99619|
|          806|Minneapolis-Saint...|      Minneapolis|   MN|    USA|44.88055| -93.21692|
|          612|Minneapolis-Saint...|      Minneapolis|   MN|    USA|44.88055| -93.21692|
|         2013|Charlotte Douglas...|        Charlotte|   NC|    USA|35.21401| -80.94313|
|         1112|Dallas

In [27]:
# Read in the airports data
airlines = spark.read.csv(['/mnt/c/Users/lokes/Downloads/airlines.csv'], header = True)

# Show the data shape
print((airlines.count(), len(airlines.columns)))

airlines.show()

(14, 2)
+---------+--------------------+
|IATA_CODE|             AIRLINE|
+---------+--------------------+
|       UA|United Air Lines ...|
|       AA|American Airlines...|
|       US|     US Airways Inc.|
|       F9|Frontier Airlines...|
|       B6|     JetBlue Airways|
|       OO|Skywest Airlines ...|
|       AS|Alaska Airlines Inc.|
|       NK|    Spirit Air Lines|
|       WN|Southwest Airline...|
|       DL|Delta Air Lines Inc.|
|       EV|Atlantic Southeas...|
|       HA|Hawaiian Airlines...|
|       MQ|American Eagle Ai...|
|       VX|      Virgin America|
+---------+--------------------+



MACHINE LEARNING

In [28]:
# filtering columns
model_data = flights.select('MONTH', 'DAY_OF_WEEK', 'AIRLINE', 'TAIL_NUMBER', 'DESTINATION_AIRPORT', 'AIR_TIME', 'DISTANCE', 'ARRIVAL_DELAY',)

# Remove missing values
model_data = model_data.filter("ARRIVAL_DELAY is not NULL and AIRLINE is not NULL and AIR_TIME is not NULL and TAIL_NUMBER is not NULL")

# rows left
model_data.count()

5714008

In [29]:
# Create is_late (label)
model_data = model_data.withColumn("is_late", model_data.ARRIVAL_DELAY > 0)

# cast
model_data = model_data.withColumn("is_late", model_data.is_late.cast("integer"))

# rename column
model_data = model_data.withColumnRenamed("is_late", 'label')

In [30]:
model_data.show(15)

+-----+-----------+-------+-----------+-------------------+--------+--------+-------------+-----+
|MONTH|DAY_OF_WEEK|AIRLINE|TAIL_NUMBER|DESTINATION_AIRPORT|AIR_TIME|DISTANCE|ARRIVAL_DELAY|label|
+-----+-----------+-------+-----------+-------------------+--------+--------+-------------+-----+
|    1|          4|     AS|     N407AS|                SEA|     169|  1448.0|          -22|    0|
|    1|          4|     AA|     N3KUAA|                PBI|     263|  2330.0|           -9|    0|
|    1|          4|     US|     N171US|                CLT|     266|  2296.0|            5|    1|
|    1|          4|     AA|     N3HYAA|                MIA|     258|  2342.0|           -9|    0|
|    1|          4|     AS|     N527AS|                ANC|     199|  1448.0|          -21|    0|
|    1|          4|     DL|     N3730B|                MSP|     206|  1589.0|            8|    1|
|    1|          4|     NK|     N635NK|                MSP|     154|  1299.0|          -17|    0|
|    1|          4| 

In [32]:
print('Labels distrubution:')
model_data.groupBy('label').count().show()

Labels distrubution:


+-----+-------+
|label|  count|
+-----+-------+
|    1|2086896|
|    0|3627112|
+-----+-------+



In [33]:
# filtering columns
model_data = flights.select('MONTH', 'DAY_OF_WEEK', 'AIRLINE', 'TAIL_NUMBER', 'DESTINATION_AIRPORT', 'AIR_TIME', 'DISTANCE', 'ARRIVAL_DELAY',)

# Remove missing values
model_data = model_data.filter("ARRIVAL_DELAY is not NULL and AIRLINE is not NULL and AIR_TIME is not NULL and TAIL_NUMBER is not NULL")

# rows left
model_data.count()

5714008

In [34]:
# Create is_late (label)
model_data = model_data.withColumn("is_late", model_data.ARRIVAL_DELAY > 0)

# cast
model_data = model_data.withColumn("is_late", model_data.is_late.cast("integer"))

# rename column
model_data = model_data.withColumnRenamed("is_late", 'label')

In [35]:
model_data.show(15)

+-----+-----------+-------+-----------+-------------------+--------+--------+-------------+-----+
|MONTH|DAY_OF_WEEK|AIRLINE|TAIL_NUMBER|DESTINATION_AIRPORT|AIR_TIME|DISTANCE|ARRIVAL_DELAY|label|
+-----+-----------+-------+-----------+-------------------+--------+--------+-------------+-----+
|    1|          4|     AS|     N407AS|                SEA|     169|  1448.0|          -22|    0|
|    1|          4|     AA|     N3KUAA|                PBI|     263|  2330.0|           -9|    0|
|    1|          4|     US|     N171US|                CLT|     266|  2296.0|            5|    1|
|    1|          4|     AA|     N3HYAA|                MIA|     258|  2342.0|           -9|    0|
|    1|          4|     AS|     N527AS|                ANC|     199|  1448.0|          -21|    0|
|    1|          4|     DL|     N3730B|                MSP|     206|  1589.0|            8|    1|
|    1|          4|     NK|     N635NK|                MSP|     154|  1299.0|          -17|    0|
|    1|          4| 

In [36]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

# Create a StringIndexer
airline_indexer = StringIndexer(inputCol="AIRLINE", outputCol="airline_index")

# Create a OneHotEncoder
airline_encoder = OneHotEncoder(inputCol="airline_index", outputCol="airline_fact")

In [37]:
# Create a StringIndexer
dest_indexer = StringIndexer(inputCol="DESTINATION_AIRPORT", outputCol="dest_index")

# Create a OneHotEncoder
dest_encoder = OneHotEncoder(inputCol="dest_index", outputCol="dest_fact")

In [38]:
# Create a StringIndexer
tail_indexer = StringIndexer(inputCol="TAIL_NUMBER", outputCol="tail_index")

# Create a OneHotEncoder
tail_encoder = OneHotEncoder(inputCol="tail_index", outputCol="tail_fact")

In [39]:
from pyspark.ml.feature import VectorAssembler

# Make a VectorAssembler of 'MONTH', 'DAY_OF_WEEK', 'AIR_TIME', 'DISTANCE', 'ARRIVAL_DELAY','AIRLINE', 'TAIL_NUMBER', 'DESTINATION_AIRPORT'
vec_assembler = VectorAssembler(inputCols=["MONTH", "DAY_OF_WEEK", "AIR_TIME", "DISTANCE", "airline_fact", "dest_fact", "tail_fact"], outputCol="features")

In [40]:
# Import Pipeline
from pyspark.ml import Pipeline

# Make the pipeline
flights_pipe = Pipeline(stages=[dest_indexer, dest_encoder, airline_indexer, airline_encoder, tail_indexer, tail_encoder, vec_assembler])

In [41]:
piped_data = flights_pipe.fit(model_data).transform(model_data)

In [42]:
train_data, test_data = piped_data.randomSplit([.7, .3])

In [43]:
print('data points(rows) in train data :',  train_data.count())
print('data points(rows) in train data :',  test_data.count())

data points(rows) in train data : 4000289


data points(rows) in train data : 1713719


In [44]:
# Import LogisticRegression
from pyspark.ml.classification import LogisticRegression

# Create a LogisticRegression Estimator
lr = LogisticRegression()

In [45]:
# Import the evaluation submodule
import pyspark.ml.evaluation as evals

# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

In [46]:
# Import the tuning submodule
import pyspark.ml.tuning as tune

# Create the parameter grid
grid1 = tune.ParamGridBuilder()

# Add the hyperparameter
grid1 = grid1.addGrid(lr.regParam, np.arange(0, .1, .01))
grid1 = grid1.addGrid(lr.elasticNetParam, [0, 1])

# Build the grid
grid1 = grid1.build()

In [47]:
# Create the CrossValidator
cv = tune.CrossValidator(estimator=lr,
               estimatorParamMaps=grid1,
               evaluator=evaluator)

In [48]:
# Call lr.fit()
best_lr = lr.fit(train_data)

# Print best_lr
print(best_lr)

22/04/18 21:22:58 WARN MemoryStore: Not enough space to cache rdd_250_0 in memory! (computed 65.0 MiB so far)
22/04/18 21:22:58 WARN BlockManager: Persisting block rdd_250_0 to disk instead.
22/04/18 21:23:00 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/04/18 21:23:00 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/04/18 21:23:50 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/04/18 21:23:50 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


LogisticRegressionModel: uid=LogisticRegression_b490e4d431c9, numClasses=2, numFeatures=5540


In [49]:
# Use the model to predict the test set
test_results = best_lr.transform(test_data)

# Evaluate the predictions
print(evaluator.evaluate(test_results))

0.6266761649417465


In [50]:
from pyspark.ml.classification import LinearSVC

lsvc= LinearSVC()

In [51]:
# Import the evaluation submodule
import pyspark.ml.evaluation as evals

# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

In [52]:
# Import the tuning submodule
import pyspark.ml.tuning as tune

# Create the parameter grid
grid2 = tune.ParamGridBuilder()

# Add the hyperparameter
grid2 = grid2.addGrid(lsvc.regParam, np.arange(0, .1, .01))


# Build the grid
grid2 = grid2.build()

In [53]:
# Create the CrossValidator
cv = tune.CrossValidator(estimator=lsvc,
               estimatorParamMaps=grid2,
               evaluator=evaluator)

In [54]:
# Call lsvc.fit()
best_lsvc = lsvc.fit(train_data)

# Print best_lr
print(best_lsvc)

22/04/18 21:26:32 WARN MemoryStore: Not enough space to cache rdd_364_0 in memory! (computed 65.0 MiB so far)
22/04/18 21:26:32 WARN BlockManager: Persisting block rdd_364_0 to disk instead.


LinearSVCModel: uid=LinearSVC_404a4d7f7f60, numClasses=2, numFeatures=5540


In [55]:
# Use the model to predict the test set
test_results = best_lsvc.transform(test_data)

# Evaluate the predictions
print(evaluator.evaluate(test_results))

0.6146774680928659


In [56]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier()


# Import the evaluation submodule
import pyspark.ml.evaluation as evals

# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

# Import the tuning submodule
import pyspark.ml.tuning as tune

# Create the parameter grid
grid3 = tune.ParamGridBuilder()

# Build the grid
grid3 = grid3.build()

# Create the CrossValidator
cv = tune.CrossValidator(estimator=dt,
               estimatorParamMaps=grid3,
               evaluator=evaluator)


# Call dt.fit()
best_dt = dt.fit(train_data)

# Print best_dt
print(best_dt)


# Use the model to predict the test set
test_results = best_dt.transform(test_data)

# Evaluate the predictions
print(evaluator.evaluate(test_results))






22/04/18 21:31:55 WARN MemoryStore: Not enough space to cache rdd_642_0 in memory! (computed 55.2 MiB so far)
22/04/18 21:31:55 WARN BlockManager: Persisting block rdd_642_0 to disk instead.
22/04/18 21:32:48 WARN MemoryStore: Not enough space to cache rdd_642_0 in memory! (computed 326.0 MiB so far)
22/04/18 21:33:20 WARN MemoryStore: Not enough space to cache rdd_642_1 in memory! (computed 55.2 MiB so far)
22/04/18 21:33:20 WARN BlockManager: Persisting block rdd_642_1 to disk instead.
22/04/18 21:34:10 WARN MemoryStore: Not enough space to cache rdd_642_1 in memory! (computed 326.0 MiB so far)


In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier()

# Import the evaluation submodule
import pyspark.ml.evaluation as evals

# Create a BinaryClassificationEvaluator
evaluator = evals.BinaryClassificationEvaluator(metricName="areaUnderROC")

# Import the tuning submodule
import pyspark.ml.tuning as tune

# Create the parameter grid
grid4 = tune.ParamGridBuilder()

# Build the grid
grid4 = grid4.build()

# Create the CrossValidator
cv = tune.CrossValidator(estimator=rf,
               estimatorParamMaps=grid4,
               evaluator=evaluator)


# Call rf.fit()
best_rf = rf.fit(train_data)

# Print best_rf
print(best_rf)


# Use the model to predict the test set
test_results = best_rf.transform(test_data)

# Evaluate the predictions
print(evaluator.evaluate(test_results))




22/03/20 14:01:23 WARN MemoryStore: Not enough space to cache rdd_689_0 in memory! (computed 55.4 MiB so far)
22/03/20 14:01:23 WARN BlockManager: Persisting block rdd_689_0 to disk instead.
22/03/20 14:02:56 WARN MemoryStore: Not enough space to cache rdd_689_0 in memory! (computed 327.3 MiB so far)
22/03/20 14:03:44 WARN MemoryStore: Not enough space to cache rdd_689_1 in memory! (computed 55.4 MiB so far)
22/03/20 14:03:44 WARN BlockManager: Persisting block rdd_689_1 to disk instead.
22/03/20 14:05:18 WARN MemoryStore: Not enough space to cache rdd_689_1 in memory! (computed 327.3 MiB so far)
22/03/20 14:06:07 WARN MemoryStore: Not enough space to cache rdd_689_2 in memory! (computed 55.4 MiB so far)
22/03/20 14:06:07 WARN BlockManager: Persisting block rdd_689_2 to disk instead.
22/03/20 14:07:41 WARN MemoryStore: Not enough space to cache rdd_689_2 in memory! (computed 327.3 MiB so far)
22/03/20 14:08:29 WARN MemoryStore: Not enough space to cache rdd_689_3 in memory! (computed 5

RandomForestClassificationModel: uid=RandomForestClassifier_fa978c1a6a00, numTrees=20, numClasses=2, numFeatures=5540


0.5418877358770496
